In [ ]:
using NBInclude
nbinclude("../src/PoissonProcessEstimation.ipynb")
import PoissonProcessEstimation

In [ ]:
# Load the data
using MAT # to load mat file
# Load some data
file = matopen("MotorHam1.mat")
rawdata = read(file, "S")
size(rawdata) #neuron by timebin by trial
# read the data into our input format, trial x (neuron) x ((spiketimes)))
hasfired = zeros(size(rawdata,1),1) # track if a neuron has fired ever
data = Array(Any, (size(rawdata,3),))
for n = 1:length(data)
    data[n] = Array(Any, (size(rawdata,1),))
    for m = 1:size(rawdata,1)        
        data[n][m] = find(x->(x>=1), rawdata[m,201:1200,n])*1e-3 # set of spiketimes between 0 and 1 sec
        if length(data[n][m]) > 0 # There was at least one trial where the neuron has fired >X times
            hasfired[m] = 1
        end
    end
end

# trim the data of neurons that are silent on every trial (no spikes whatsoever within the whole dataset)
for n = 1:length(data)
    data[n] = data[n][find(x->x==1, hasfired)]
end
        

In [ ]:
# Set up kernels
KernList = [MLKernels.GaussianKernel(100.0), MLKernels.GaussianKernel(100.0), MLKernels.GaussianKernel(100.0), MLKernels.GaussianKernel(100.0)]

In [ ]:
# Initialise params
# Full dataset
θ = PoissonProcessEstimation.create_rand_params(length(data)-10,length(data[1]),length(KernList),[20,20,20,20]);
# Use last 10 trials to evaluate predictive likelihood


#Reduced dataset
#θ = PoissonProcessEstimation.create_rand_params(50,20,length(KernList),[20,20,20,10]);

# Symmetry breaking of C
θ.C = θ.C + randn(size(θ.C))*0.05;

In [ ]:
θ_orig = θ
θ_opt = deepcopy(θ)
C, α, u, N, M, D, J = PoissonProcessEstimation.name_params(θ_opt);
println("N=$(N), M=$(M), D=$(D)")

In [ ]:
# Run optimisation
@time Ktu, dtKtu, ddtKtu, Kuu, I = 
    PoissonProcessEstimation.optimise!(data, θ_opt, KernList, η_RKHS=1e2, η_Cm=1e1, num_iters = 25, verbose=false, compute_likelihood=false);

In [ ]:
#using JLD
save("MotorHam1_results_kernel_all_likelihood.jld", "theta_opt", θ_opt, "data", data, "theta", θ, "KernList", KernList)

In [ ]:
PoissonProcessEstimation.plot_params(data, θ_opt, θ_orig, KernList, num_neur=35, trials=1:100)

In [ ]:
ll, all_ll, rate_at_spikes, Λ = PoissonProcessEstimation.log_likelihood(data, θ_opt, KernList);

In [ ]:
ll

# Test predictive log_likelihood

In [ ]:
#using JLD
#θ_opt, data, θ, KernList= load("MotorHam1_results_kernel_all_likelihood.jld", "theta_opt", "data",  "theta",  "KernList");

In [ ]:
# As an alternative model, just predict the firing rate to be homogeneous and the average of previous trials
θ_flat = PoissonProcessEstimation.RKHS_params(zeros(size(θ_opt.C)), θ_opt.α, θ_opt.u, log(mean(convert(Array{Float64,2},hcat(I...)),2))) # Predict with average the observed firing rate;


In [ ]:
# Compute predictive likelihoods given the m-th neuron is unobserved on the nth trial
predLL = zeros(2,10,M)
for n = 1:10
    for m = 1:M
        predLL[:, n, m] =
        [PoissonProcessEstimation.predictive_log_likelihood(N-10+n, m, data, deepcopy(θ_opt), KernList),
            PoissonProcessEstimation.predictive_log_likelihood(N-10+n, m, data, deepcopy(θ_flat), KernList)]
    end
end